In [3]:
import numpy as np

## Example data 

In [53]:
numbers = (7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1)

boards = np.ma.array([
    [22, 13, 17, 11,  0,  8,  2, 23,  4, 24, 21,  9, 14, 16,  7,  6, 10,  3, 18,  5,  1, 12, 20, 15, 19],
    [3, 15,  0,  2, 22,  9, 18, 13, 17,  5, 19,  8,  7, 25, 23, 20, 11, 10, 24,  4, 14, 21, 16, 12,  6],
    [14, 21, 17, 24,  4, 10, 16, 15,  9, 19, 18,  8, 23, 26, 20, 22, 11, 13,  6,  5,  2,  0, 12,  3,  7]
], mask=False).reshape([-1, 5, 5])


## Masked Arrays

See: https://numpy.org/doc/stable/reference/maskedarray.html

`board` is a numpy masked array, with the maks initially set to all False values. This means no values are masked

In [54]:
print(boards)

[[[22 13 17 11 0]
  [8 2 23 4 24]
  [21 9 14 16 7]
  [6 10 3 18 5]
  [1 12 20 15 19]]

 [[3 15 0 2 22]
  [9 18 13 17 5]
  [19 8 7 25 23]
  [20 11 10 24 4]
  [14 21 16 12 6]]

 [[14 21 17 24 4]
  [10 16 15 9 19]
  [18 8 23 26 20]
  [22 11 13 6 5]
  [2 0 12 3 7]]]


Given a number we can mask all values with that number in one call. The default for `copy` is True, but here we'll make the modification to the array in place to mark the cells with values from `[7, 4, 9, 5, 11)]`:

In [55]:
for n in numbers[:5]:
    np.ma.masked_equal(boards, n, copy=False)

print(boards)

[[[22 13 17 -- 0]
  [8 2 23 -- 24]
  [21 -- 14 16 --]
  [6 10 3 18 --]
  [1 12 20 15 19]]

 [[3 15 0 2 22]
  [-- 18 13 17 --]
  [19 8 -- 25 23]
  [20 -- 10 24 --]
  [14 21 16 12 6]]

 [[14 21 17 24 --]
  [10 16 15 -- 19]
  [18 8 23 26 20]
  [22 -- 13 6 --]
  [2 0 12 3 --]]]


Once enough numbers are called we will have a winner which is noted by the entire row being masked:

In [56]:
for n in numbers[5:12]:
    np.ma.masked_equal(boards, n, copy=False)

print(boards)

[[[22 13 -- -- --]
  [8 -- -- -- --]
  [-- -- -- 16 --]
  [6 10 3 18 --]
  [1 12 20 15 19]]

 [[3 15 -- -- 22]
  [-- 18 13 -- --]
  [19 8 -- 25 --]
  [20 -- 10 -- --]
  [-- -- 16 12 6]]

 [[-- -- -- -- --]
  [10 16 15 -- 19]
  [18 8 -- 26 20]
  [22 -- 13 6 --]
  [-- -- 12 3 --]]]


To find this board, we can ask where `all()` numbers are masked on a certain axis (2 for rows, 1 for columns), and discover that board at index 3 (row 0) is the winnder. The `mask` on a masked array is simply a booleans array, so we can pass it to `argwhere` to find where the True value is. It's possible more than one board wins, which is why this returns a 2-dimensional array. We are assuming for this that won't happen, so we grab the board with `boards[row_win[0][0]]`

In [57]:
print(boards.all(axis=2))

row_win = np.argwhere(boards.all(axis=2).mask)
print("Winning index:", row_win)
winning_board = boards[row_win[0][0]]
print("Winning board:", winning_board, sep="\n")

[[True True True True True]
 [True True True True True]
 [-- True True True True]]
Winning index: [[2 0]]
Winning board:
[[-- -- -- -- --]
 [10 16 15 -- 19]
 [18 8 -- 26 20]
 [22 -- 13 6 --]
 [-- -- 12 3 --]]


Now the payoff with masked arrays. Since we masked the values as they were called, we can just call `sum()` to get the sum of unmasked values, which is conveneiently what the problem asks for:

In [29]:
winning_board.sum()

188

## Main Solution

In [48]:
import os
from pathlib import Path
import numpy as np


FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day4.txt'

with open(FOLDER / in_file) as f:
    numbers = np.loadtxt([next(f)], delimiter=',', dtype=int)
    boards = np.ma.array(np.loadtxt(f, dtype=int).reshape(-1, 5, 5), mask=False)

boards[0]

masked_array(
  data=[[57, 7, 8, 38, 31],
        [17, 96, 5, 12, 18],
        [58, 45, 81, 89, 4],
        [73, 51, 93, 32, 10],
        [74, 50, 26, 0, 24]],
  mask=[[False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False]],
  fill_value=999999)

## Part One

In [49]:
def solution_one(boards, numbers):
    for n in numbers:
        np.ma.masked_equal(boards, n, copy=False)

        col_win = np.argwhere(boards.all(axis=1).mask)
        if col_win.size:
            board = boards[col_win[0][0]]
            print("winning board", board, sep='\n')
            return board.sum() * n

        row_win = np.argwhere(boards.all(axis=2).mask)
        if row_win.size:
            board = boards[row_win[0][0]]
            print("winning board", board, sep='\n')
            return board.sum() * n

In [50]:
print("Solution 1:", solution_one(boards.copy(), numbers))

winning board
[[86 -- 77 -- 87]
 [79 -- 52 17 20]
 [30 -- 48 -- --]
 [25 -- 13 9 47]
 [45 -- 97 15 59]]
Solution 1: 58838


## Part Two

This is the same idea. The difference is that we now keep track of the boards still playing. This makes it possible to only ask for winners from currently active players by using `playing` as a mask:

`np.argwhere(boards.all(axis=1).mask & playing[:, np.newaxis])`

Then keep track of the last solution.

Since it's possible for more than board to win in a round, mark more than one board as no longer playing. `numpy.put()` is good for this:

`np.put(playing, col_win[:,0], False)`

The problem doesn't make sense if more than one board can be the *last* board to win, so this ignores that possibility.


In [64]:
def solution_two(boards, numbers):
    playing = np.ones(boards.shape[0], dtype=bool)
    last_solution = None

    for n in numbers:
        np.ma.masked_equal(boards, n, copy=False)
        col_win = np.argwhere(boards.all(axis=1).mask & playing[:, np.newaxis])
        if col_win.size:
            board = boards[col_win[0][0]]
            np.put(playing, col_win[:,0], False)
            last_solution = board.sum() * n

        row_win = np.argwhere(boards.all(axis=2).mask & playing[:, np.newaxis])
        if row_win.size:
            board = boards[row_win[0][0]]
            np.put(playing, row_win[:, 0], False)
            last_solution =  board.sum() * n

    return last_solution 

solution_two(boards.copy(), numbers)


6256